In [1]:
import pandas as pd

In [2]:
train_data=pd.read_csv('titanic_data.csv')

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
import numpy as np

In [6]:
(np.unique(train_data['PassengerId'].values).size,np.unique(train_data['PassengerId'].values).size==train_data.shape[0])

(891, True)

In [7]:
train_data.set_index(['PassengerId'],inplace=True)

In [8]:
train_data.head(0)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,


In [9]:
import re

In [10]:
patt=re.compile('\s(\S+\.)')

In [11]:
titles=np.array([re.search(patt,i)[1]for i in train_data['Name'].values])

In [12]:
print('Unique titles ({}) : \n{}'.format(np.unique(titles).shape[0],np.unique(titles)))

Unique titles (17) : 
['Capt.' 'Col.' 'Countess.' 'Don.' 'Dr.' 'Jonkheer.' 'Lady.' 'Major.'
 'Master.' 'Miss.' 'Mlle.' 'Mme.' 'Mr.' 'Mrs.' 'Ms.' 'Rev.' 'Sir.']


In [13]:
print('')

In [14]:
print('Number of titles that are NaN/Null : {}'.format(pd.isnull(titles).sum()))

Number of titles that are NaN/Null : 0


In [15]:
train_data=train_data.assign(Title=titles)

In [16]:
train_data=train_data.drop('Name',axis=1)

In [17]:
train_data['Title'].groupby(train_data['Title']).size()

Title
Capt.          1
Col.           2
Countess.      1
Don.           1
Dr.            7
Jonkheer.      1
Lady.          1
Major.         2
Master.       40
Miss.        182
Mlle.          2
Mme.           1
Mr.          517
Mrs.         125
Ms.            1
Rev.           6
Sir.           1
Name: Title, dtype: int64

In [22]:
train_data['Title'] = train_data['Title'].replace('Mlle.','Miss.')
train_data['Title'] = train_data['Title'].replace('Ms.','Miss.')  
train_data['Title'] = train_data['Title'].replace('Mme.','Mrs.')
train_data['Title'] = train_data['Title'].replace(['Capt.','Col.','Major.'],'Army.')
train_data['Title'] = train_data['Title'].replace(['Countess.','Don.','Jonkheer.','Lady.','Sir.'],'Noble.')

print('Number of passengers:\n{}'.format(train_data['Title'].\
                                         groupby(train_data['Title']).size()))
print('')
print('Average survival:\n{}'.format(train_data[['Title','Survived']].\
                                     groupby(train_data['Title']).mean()))

Number of passengers:
Title
Army.        5
Dr.          7
Master.     40
Miss.      185
Mr.        517
Mrs.       126
Noble.       5
Rev.         6
Name: Title, dtype: int64

Average survival:
         Survived
Title            
Army.    0.400000
Dr.      0.428571
Master.  0.575000
Miss.    0.702703
Mr.      0.156673
Mrs.     0.793651
Noble.   0.600000
Rev.     0.000000


In [23]:
train_data = train_data.drop('Ticket', axis=1)
train_data.head(0)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,


In [24]:
def getCabinCat(cabin_code):
    if pd.isnull(cabin_code):
        cat = 'None' # Use a string so that it is clear that this is 
                     # a category on its own
    else:
        cat = cabin_code[0]
    return cat

In [25]:
cabin_cats = np.array([getCabinCat(cc) for cc in train_data['Cabin'].values])


In [26]:
train_data = train_data.assign(Cabin_cat=cabin_cats)
train_data = train_data.drop('Cabin', axis=1)
train_data.head(0)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_cat
PassengerId,,,,,,,,,,


In [27]:
print('Number of passengers:\n{}'.format(train_data['Cabin_cat'].\
                                groupby(train_data['Cabin_cat']).size()))

Number of passengers:
Cabin_cat
A        15
B        47
C        59
D        33
E        32
F        13
G         4
None    687
T         1
Name: Cabin_cat, dtype: int64


In [28]:
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_cat
PassengerId,,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S,Mr.,None
2,1,1,female,38.0,1,0,71.2833,C,Mrs.,C
3,1,3,female,26.0,0,0,7.9250,S,Miss.,None
4,1,1,female,35.0,1,0,53.1000,S,Mrs.,C
5,0,3,male,35.0,0,0,8.0500,S,Mr.,None
...,...,...,...,...,...,...,...,...,...,...
887,0,2,male,27.0,0,0,13.0000,S,Rev.,None
888,1,1,female,19.0,0,0,30.0000,S,Miss.,B
889,0,3,female,NaN,1,2,23.4500,S,Miss.,None
